In [19]:
# import packages
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import Bio
from Bio.PDB import PDBParser
from Bio.PDB.DSSP import DSSP
import statsmodels.api as sm
from scipy.optimize import curve_fit
import pylab
from sklearn.cluster import AffinityPropagation
from Levenshtein import distance



In [13]:
p = PDBParser()
structure = p.get_structure("6io9", "C:/Users/marik/Downloads/6io9.pdb")
model = structure[0]
#dssp = DSSP(model, "C:/Users/marik/Downloads/6io9.pdb", dssp='mkdssp')


In [2]:
# import data from computer
path: str = './data/' #folder where files are stored
prokaryotes: pd.DataFrame = pd.read_csv(os.path.join(path, "prokaryotes_unique_prot.csv"), dtype=str)   # imports file from the folder as species, based on name
prokaryotes_all: pd.DataFrame = pd.read_csv(os.path.join(path, "prokaryotes_all.csv"), dtype=str) # imports 3rd file from the folder, based on name

prokaryotes_all['temperature'] = prokaryotes_all['temperature'].astype('float64')
prokaryotes_all['fold_change'] = prokaryotes_all['fold_change'].astype('float64')
prokaryotes_all['Gene Ontology IDs'] = prokaryotes_all['Gene Ontology IDs'].astype('str')

In [21]:
GO_terms = []
current_term = ""

with open(os.path.join(path, "GO_annotation_list.obo"), 'r') as f:
    for line in f:
      line = line.strip()  
      if line.startswith('[Term]'):
        if current_term:
          GO_terms.append(current_term.strip())  
        current_term = ""
      else:
        current_term += line + "\n"  
    if current_term:
      GO_terms.append(current_term.strip())

GO_terms = np.asarray(GO_terms)   

In [24]:

lev_similarity = -1*np.array([[distance(w1,w2) for w1 in GO_terms] for w2 in GO_terms])

affprop = AffinityPropagation(affinity="precomputed", damping=0.5)
affprop.fit(lev_similarity)
for cluster_id in np.unique(affprop.labels_):
    exemplar = GO_terms[affprop.cluster_centers_indices_[cluster_id]]
    cluster = np.unique(GO_terms[np.nonzero(affprop.labels_==cluster_id)])
    cluster_str = ", ".join(cluster)
    print(" - *%s:* %s" % (exemplar, cluster_str))

KeyboardInterrupt: 